# PODMAN ADMINISTRATION TOOLS

**Podman** is a daemonless, open source, Linux native tool designed to make it easy to 
- find, 
- run, 
- build, 
- share and 
- deploy 

applications using [Open Containers Initiative (OCI)](https://www.opencontainers.org/) Containers and Container Images.

# Basics

## Configuration files

Podman has its built-in defaults but defines three locations for its configuration files to be stored:

- `/usr/share/containers/containers.conf` — Where a distribution can define the changes the distribution likes to use
- `/etc/containers/containers.conf` — Where they can set up system overrides
- `$HOME/.config/containers/containers.conf` — Can be specified only in rootless mode

## Storage dir

- `/var/lib/containers/storage/` - storage directory (accessible under root only)

## Podman Basic commands

|key|description|
|-|-|
|`podman --help`|list all the commands|
|**Build a container**||
|||
|**Volume**||

### Get an image

|key|description|
|-|-|
|**Get an image**||
|`podman search <image_name>`|search for image, [`registries.conf`](#Registries) is needed|
|`skopeo inspect -- raw docker://<link> \| jq`|inspect the image without downloading it, in json view|
|`skopeo inspect docker://<link> \| jq .Layers`|show the layers of the repository|
|`podman pull <image_name>` or `<link> [:tag]`|get the copy of the the repository to the container host|

### List data

|key|description|
|-|-|
|**List data**||
|`podman images`|list all existing images on the system|
|`podman images -q`|list ids of the images|
|`podman ps [params]`|list running Docker containers|
|`-a`|show all the containers including stopped ones|
|`-q`|show the ids of the containers|
|`-s` or `--size`|show the amount of disk space currently used for each container|

### Create/start, run, stop

|key|description|
|-|-|
|**Create a container**||
|`podman create`|creates but does not start the container|
|**Run a container**||
|`podman start <container_name>`|starting a created or already existing but stopped container|
|`--all` (note: `-a` is for different command in `start`)|starts all of the stopped containers in container storage|
|`podman run -ti [params] <container_image>`||
|`-i` or `--interactive`|Keep STDIN open even if not attached|
|`-t` or `--tty`|Allocate a pseudo-TTY|
|`--name <container_name>`|(not recommended) add a name to the container|
|`-d` or `--detach`|run the container in the background|
|`--rm`|remove the container as soon as it exits, freeing up all of the container’s storage|
|`-e <parameter>`|add an environmental variable (`-e TZ=Europe/Yekaterinburg`)|
|`-p <host_port>:<container_port>` or `--publish`|container port should be `EXPOSE`d in the Docker file|
|`-v <host_dir_path>:<docker_dir_path>`|save files into the host directory|
|`--user USERNAME`|run the container as a specific user defined in the image; by default, Podman will run the container as root, unless the container image specifies a default user|
|`podman port <container_name>`|which ports are bound to a container|
|**Stop a container**||
|`podman stop <container_name>` or `<id>`|forcibly stop the container|
|`-t` or `--time`|`-t 0` sends the `SIGKILL` without waiting for the container to stop|
|`-a` or `--all`|stop all the containers|
|**Get into shell**||
|`podman exec -ti <container_id_or_name> bash` or `/bin/bash`||

### Remove

|key|description|
|-|-|
|**Remove containers**||
|`podman rm <container_name>` or `<id>`|remove a container|
|`podman rm $(podman ps -qa)` or `-a`|remove all the containers using ids|
|**Remove images**||
|`podman rmi <container_image>`|remove the image from the container host|
|`podman rmi $(docker images -q)` or `-a`|remove all the images from the host|

### Push/pull

|key|description|
|-|-|
|**Share the container**||
|`podman login <registry_name>`|login to the registry, like docker.io|
|`podman build -t <username>/<image_name>`||
|`podman push <username>/<container_name>`|publish the container|
|`podman search <username>/<container_name>`|now you can find and pull it|

# Administration commands

|key|description|
|-|-|
|**Security**||
|`podman trust inspect <container_image_name>`|show the signers and the expiration details for the image|
|**Administration**||
|`podman info`|see the technical info|
|`podman inspect <tag>`|"inspect" a running container for metadata and details about itself|
|`podman logs <container_id>`|view the container's logs|
|`podman top <container_id>`|view the container's pids|
|`sudo podman container checkpoint <container_id>`| a container can later be restored and continue running at exactly the same point in time as the checkpoint (container must be created as root)|
|`sudo podman container restore <container_id>`|restoring a container is only possible for a previously checkpointed container; the restored container will continue to run at exactly the same point in time it was checkpointed|
|||
|||

## Migrate the container

To live migrate a container from one host to another the container is 
- checkpointed on the source system of the migration, 
- transferred to the destination system and then 
- restored on the destination system. 

When transferring the checkpoint, it is possible to specify an output-file.

On the source system:

```sh
sudo podman container checkpoint <container_id> -e /tmp/checkpoint.tar.gz
scp /tmp/checkpoint.tar.gz <destination_system>:/tmp
```

On the destination system:

```sh
sudo podman container restore -i /tmp/checkpoint.tar.gz
```

After being restored, the container will answer requests again as it did before checkpointing. This time the container will continue to run on the destination system.

## Managing pods

|key|description|
|-|-|
|`podman pod --help`||
|**Create a pod**||
|`podman pod create --name <pod_name>`|creates a new and empty pod|
|`podman pod ls`|list all created pods|
|`podman ps -a --pod`|list all the containers in the pod|
|**Run pods**||
|`podman run -dt --pod <pod_name> <container_image_name> -p <host_port>:<container_port>`|run the image within the pod|
|`podman rn -dt --pod new:<pod_name> -p <host_port>:<container_port> <container_image_name>`|create and run a new container within the pod|
|`podman pod start <pod_name>`||
|`podman pod stop <pod_name>`||
|**Run and stop containers from the pod**||
|`podman stop <container_name_or_id>`||
|`podman start <container_name_or_id>`||
|**Remove**||
|`podman pod rm <pod_name>`|remove the pod (first stop it)|
|`podman pod rm <container_name_or_id>`|remove a container from the pod (first stop it)|

## Kubernetes YAML Manifest

|key|description|
|-|-|
|`podman generate kube <pod_name>`||
|`podman generate kube <pod_name> >> <file_name>.yaml`||
|`podman play kube <file_name>.yaml`|creates a new pod bases on the k8s file provided|
|||
|||
|||

## <a name='Registries'>Registries</a>

```sh
mkdir /home/$USER/.config/containers/

vi /home/$USER/.config/containers/registries.conf

unqualified-search-registries = ['docker.io', 'ghrc.io', 'quay.io']
:wq
```

# Deeper

## Network handling

In [2]:
!man podman-run

podman-run(1)               General Commands Manual              podman-run(1)

NAME
       podman-run - Run a command in a new container

SYNOPSIS
       podman run [options] image [command [arg ...]]

       podman container run [options] image [command [arg ...]]

DESCRIPTION
       Run  a process in a new container. podman run starts a process with its
       own file system, its own networking, and its own isolated process tree.
       The  image  which starts the process may define defaults related to the
       process that will be run in the container, the  networking  to  expose,
       and  more, but podman run gives final control to the operator or admin‐
       istrator who starts the container from the image. For that reason  pod‐
       man run has more options than any other Podman command.

       If the image is not already loaded then podman run will pull the image,
       and all image dependencies, from the repository in the same way running
       podman pull image

### `/etc/hosts`

```sh
podman run -d -p 8080:8080 --name myapp registry.access.redhat.com/ubi8/httpd-2

cat /etc/hosts
```
```
127.0.0.1       localhost
127.0.1.1       debian-laptop
::1     localhost ip6-localhost ip6-loopback
ff02::1 ip6-allnodes
ff02::2 ip6-allrouters
192.168.1.94    host.containers.internal
10.0.2.100      09d65cf66423 myapp
```

ChatGPT:

The output of the command `cat /etc/hosts` lists the mapping of hostnames to IP addresses in the container. Let's break down the output:

- `127.0.0.1       localhost`: This line maps the hostname "localhost" to the loopback IP address 127.0.0.1, which is the standard loopback interface address.

- `127.0.1.1       debian-laptop`: This line typically identifies the machine's hostname and is a common setup in Linux systems.

- `::1     localhost ip6-localhost ip6-loopback`: This line maps the IPv6 loopback address "::1" to hostnames "localhost", "ip6-localhost", and "ip6-loopback".

- `ff02::1 ip6-allnodes`: This line maps the IPv6 multicast address "ff02::1" to the hostname "ip6-allnodes".

- `ff02::2 ip6-allrouters`: This line maps the IPv6 multicast address "ff02::2" to the hostname "ip6-allrouters".

- `192.168.1.94    host.containers.internal`: This line maps the IP address 192.168.1.94 to the hostname "host.containers.internal". This is a common mapping used in container networking setups.

- `10.0.2.100      09d65cf66423 myapp`: This line maps the IP address 10.0.2.100 to the hostname "09d65cf66423" and "myapp". The hostname "myapp" is the name given to your container using the --name option in the podman run command. The hostname "09d65cf66423" seems to be generated from the container ID.

This file is used to map hostnames to IP addresses, and it's especially important in containerized environments where containers may have their own networking configurations.

**Why are there two vlans for one container?**

There could be several reasons why there are two VLANs for one container:

- **Isolation and Security**: The use of two VLANs can provide an extra layer of security and isolation for the container. For example, one VLAN could be dedicated to internal communication within the container, while the other VLAN could be used for external communication with other systems.

- **Network Segmentation**: Two VLANs can be used to segment the network traffic within the container, allowing for better organization and management of the traffic flow.

- **Compliance Requirements**: Certain industries or organizations may have compliance requirements that mandate the use of multiple VLANs for specific purposes, such as separating sensitive data from non-sensitive data.

- **Application Requirements**: The container may have specific networking requirements or dependencies that necessitate the use of multiple VLANs to support its functionality.

Overall, the use of two VLANs for one container allows for more granular control and management of the network traffic and can help to improve security and performance.

### `/etc/resolv.conf`

```
cat /etc/resolv.conf
search localnet
nameserver 10.0.2.3
nameserver 192.168.1.1
```

# Postgresql image

ChatGPT:

Using an official Postgres container image is often preferable to creating your own from scratch for several reasons:

- **Security:** Official container images are maintained and updated by the Postgres developers, so they often include security patches and updates. By using an official image, you can benefit from the expertise and resources of the community supporting it, reducing the risk of potential vulnerabilities.

- **Efficiency:** An official container image is typically optimized for performance and resource usage, saving you the time and effort required to configure and tune a custom container from scratch.

- **Compatibility:** Official container images are designed to work seamlessly with other tools and services in the Docker and Kubernetes ecosystems. By using the official image, you can be confident that it will integrate smoothly with other components in your infrastructure.

- **Reliability:** Official container images are thoroughly tested and validated, reducing the likelihood of unexpected issues or bugs. They are also widely used, which means that you can rely on a large community for support and troubleshooting.

In summary, using an official Postgres container image can provide greater security, efficiency, compatibility, and reliability compared to creating and maintaining your own container from scratch.

```sh
podman pull docker.io/bitnami/postgresql:15
podman run -ti --rm -e POSTGRES_PASSWORD='password' -d -p 5444:5432 -v <host_dir>:<pd_dir> postgres:15

podman exec -ti <container_name> psql -U postgres
# or
podman exec -ti $(podman ps -qf "name=my-postgres") psql -U postgres
```

## Postgresql-container on the server

### Prepare the directory

```sh
port_number=1945
container_path=/home/$USER/containers/postgresql

mkdir -p $container_path
touch $container_path/Containerfile

# open a port for the future container
su -
ufw limit $port_number
ufw reload
```

### `Containerfile`

```
FROM docker.io/library/postgres:15

COPY postgresql.conf /var/lib/postgresql/data
COPY pg_hba.conf /var/lib/postgresql/data
```

### Run on the server

```sh
podman run --rm -e POSTGRES_PASSWORD=67890 --name postgresql-container -p $port_number:5432 -d -v /home/$USER/containers/postgresql/data/:/etc/postgresql postgresql-15
```

```sh

# port for the container
port_number=1945

# build the image
podman build -t postgresql-15 .

# run the container
podman run --rm -e POSTGRES_PASSWORD=67890 --name postgresql-container -p $port_number:5432 -d postgresql-15

# we need to create user and database
podman exec -ti postgresql-container psql -U postgres
```

```sql
CREATE ROLE person LOGIN PASSWORD '12345';
CREATE DATABASE test WITH OWNER person;
```

ChatGPT:

When you forward the server port `5432` to a different host port for the PostgreSQL container, it essentially achieves the same result as changing the server port directly. By doing so, you are essentially telling the container to listen on a different port (the one you specified) instead of the default port 5432, effectively achieving the same level of security.

### Connect from the client

```sh
port_number=1945
psql -p $port_number -h 192.168.1.62 -d test -U person
```